In [1]:
import pandas as pd
import xml.etree.ElementTree as ET
import csv

In [2]:
def is_in_terms_list(term: ET.Element, terms_list: list):
    for cur in terms_list:
        if term[1].text == cur:
            return True
    return False

In [4]:
def correct_description(description):
    description = description.replace('\n','')
    description = description.rstrip(' ')
    return description

In [16]:
def xml_parsing(input_file_name, output_file_name, terms_list, delimiter):
    found_count = 0
    terms_num = len(terms_list)
    with open(output_file_name, mode="w", encoding='utf-8') as output_file:
        file_writer = csv.writer(output_file, delimiter = delimiter, lineterminator="\r")
        file_writer.writerow(["Sep = \\t"])
        file_writer.writerow(["MeSH Term", "Description", "Synonyms"])
        for event, elem in ET.iterparse(input_file_name):
            if elem.tag == 'DescriptorRecord':
                descriptor_record_elem = elem
                concept_list_elem = descriptor_record_elem.find('ConceptList')
                for concept in concept_list_elem.findall('Concept'):
                    scope_note_elem = concept.find('ScopeNote')
                    term_list_elem = concept.find('TermList')
                    MeSH_term = None
                    description = None
                    synonyms = []
                    found = False
                    for term in term_list_elem.findall('Term'):
                        synonyms.append(term[1].text)
                        if is_in_terms_list(term, terms_list):
                            found = True
                            MeSH_term = term[1].text
                            synonyms.pop()
                            if scope_note_elem is not None:
                                description = correct_description(scope_note_elem.text)
                            else:
                                description = 'Description is missing'

                    if found:
                        found_count += 1
                        if len(synonyms) == 0:
                            synonyms = ['Synonyms are missing','']
                        # print('term =', MeSH_term)
                        # print('Description = ',description)
                        # print('Synonyms: ', synonyms)
                        file_writer.writerow([MeSH_term, description, *synonyms])

                    if found_count == terms_num:
                            break
            else:
                continue

In [6]:
data_1 = pd.read_csv('data1.csv', delimiter='\t')
disease_list = data_1['MeSH Disease Term'].to_list()
xml_parsing("desc2022.xml","data_1_disease_description_synonyms_raw.csv", disease_list, '\t')

In [20]:
data_2 = pd.read_csv('data2.csv', delimiter='\t')
symptom_list = data_2['MeSH Symptom Term'].to_list()
xml_parsing("desc2022.xml","data_2_symptom_description_synonyms_raw.csv", symptom_list, '\t')